In [21]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import math

In [2]:
anime = pd.read_csv("data/anime.csv")
anime.head()

,title,mediaType,eps,duration,ongoing,startYr,finishYr,sznOfRelease,description,studios,tags,contentWarn,watched,watching,wantWatch,dropped,rating,votes
0,Fullmetal Alchemist: Brotherhood,TV,64.0,NaN,False,2009.0,2010.0,Spring,The foundation of alchemy is based on the law ...,['Bones'],"['Action', 'Adventure', 'Drama', 'Fantasy', 'M...","['Animal Abuse', 'Mature Themes', 'Violence', ...",103707.0,14351,25810,2656,4.702,86547.0
1,your name.,Movie,1.0,107.0,False,2016.0,2016.0,NaN,Mitsuha and Taki are two total strangers livin...,['CoMix Wave Films'],"['Drama', 'Romance', 'Body Swapping', 'Gender ...",[],58831.0,1453,21733,124,4.663,43960.0
2,A Silent Voice,Movie,1.0,130.0,False,2016.0,2016.0,NaN,"After transferring into a new school, a deaf g...",['Kyoto Animation'],"['Drama', 'Shounen', 'Disability', 'Melancholy...","['Bullying', 'Mature Themes', 'Suicide']",45892.0,946,17148,132,4.661,33752.0
3,Haikyuu!! Karasuno High School vs Shiratorizaw...,TV,10.0,NaN,False,2016.0,2016.0,Fall,"Picking up where the second season ended, the ...",['Production I.G'],"['Shounen', 'Sports', 'Animeism', 'School Club...",[],25134.0,2183,8082,167,4.660,17422.0
4,Attack on Titan 3rd Season: Part II,TV,10.0,NaN,False,2019.0,2019.0,Spring,The battle to retake Wall Maria begins now! Wi...,['Wit Studio'],"['Action', 'Fantasy', 'Horror', 'Shounen', 'Da...","['Cannibalism', 'Explicit Violence']",21308.0,3217,7864,174,4.650,15789.0


In [3]:
#Clean studios column from "list of strings" to a string
anime['studios'] = anime['studios'].str.replace("[", "")
anime['studios'] = anime['studios'].str.replace("]", "")
anime['studios'] = anime['studios'].str.lstrip("'").str.rstrip("'")
#final_username = username.replace("_", "")


,title,mediaType,eps,duration,ongoing,startYr,finishYr,sznOfRelease,description,studios,tags,contentWarn,watched,watching,wantWatch,dropped,rating,votes
14528,Ore no Sora Keiji-hen,OVA,2.0,44.0,False,1991.0,1992.0,NaN,NaN,Studio A.P.P.P.,"['Action', 'Drama', 'Police', 'Based on a Manga']",[],14.0,2,35,1,NaN,NaN
14529,Sakkai Eightraid,Web,NaN,NaN,False,NaN,NaN,NaN,The story takes place in a universe where coun...,Digital Network Animation,"['Action', 'Sci Fi', 'Original Work']",[],0.0,0,851,0,NaN,NaN
14530,"Moshi Moshi, Terumi desu.",Other,30.0,3.0,False,2018.0,2018.0,NaN,Terumi works for a telephone company that has ...,,"['Comedy', 'Short Episodes', 'Work Life', 'Bas...",[],7.0,2,68,1,NaN,NaN
14531,Neko Konogoro,TV,21.0,1.0,True,2016.0,NaN,Spring,NaN,Kachidoki Studio,"['Animal Protagonists', 'Cats', 'Non-Human Pro...",[],NaN,9,40,4,NaN,NaN
14532,Something Great Chizu ni Nai Machi,Movie,1.0,NaN,False,2006.0,2006.0,NaN,NaN,,['Slice of Life'],[],5.0,1,15,1,NaN,NaN
14533,Taeng-gu & Ulasyong,TV,26.0,NaN,False,2001.0,2001.0,NaN,NaN,,"['Action', 'Comedy', 'Mecha', 'Korean Animation']",[],8.0,1,12,1,NaN,NaN
14534,Go! Go! Senkyo,Web,10.0,3.0,False,2016.0,2016.0,NaN,NaN,,"['Animal Protagonists', 'Anthropomorphic', 'Da...",[],16.0,2,12,0,NaN,NaN
14535,Violet Evergarden Movie,Movie,NaN,NaN,False,2020.0,2020.0,NaN,NaN,Kyoto Animation,"['Drama', 'Based on a Light Novel']",[],0.0,0,5623,0,NaN,NaN
14536,Nyuru Nyuru!! Kakusen-kun 2nd Season,TV,13.0,5.0,False,2015.0,2015.0,Spring,NaN,DLE,"['Family Friendly', 'Short Episodes']",[],9.0,1,13,1,NaN,NaN
14537,Manga Furusato Mukashibanashi,TV,26.0,NaN,False,1976.0,1976.0,NaN,NaN,,"['Fantasy', 'Episodic', 'Family Friendly', 'Ja...",[],7.0,0,10,0,NaN,NaN


In [5]:
#Clean list function
def clean_list(col):
    col = col.str.replace("[", "")
    col = col.str.replace("]", "")
    col = col.str.replace("'", "")
    col = col.str.split(",")
    return col

anime['tags'] = clean_list(anime['tags'])
anime['contentWarn'] = clean_list(anime['contentWarn'])



# anime['tags'] = anime['tags'].str.replace("[", "")
# anime['tags'] = anime['tags'].str.replace("]", "")
# anime['tags'] = anime['tags'].str.replace("'", "")
# anime['tags'] = anime['tags'].str.split(",")

,title,mediaType,eps,duration,ongoing,startYr,finishYr,sznOfRelease,description,studios,tags,contentWarn,watched,watching,wantWatch,dropped,rating,votes
0,Fullmetal Alchemist: Brotherhood,TV,64.0,NaN,False,2009.0,2010.0,Spring,The foundation of alchemy is based on the law ...,Bones,"[Action, Adventure, Drama, Fantasy, Myster...","[Animal Abuse, Mature Themes, Violence, Dom...",103707.0,14351,25810,2656,4.702,86547.0
1,your name.,Movie,1.0,107.0,False,2016.0,2016.0,NaN,Mitsuha and Taki are two total strangers livin...,CoMix Wave Films,"[Drama, Romance, Body Swapping, Gender Bend...",[],58831.0,1453,21733,124,4.663,43960.0
2,A Silent Voice,Movie,1.0,130.0,False,2016.0,2016.0,NaN,"After transferring into a new school, a deaf g...",Kyoto Animation,"[Drama, Shounen, Disability, Melancholy, M...","[Bullying, Mature Themes, Suicide]",45892.0,946,17148,132,4.661,33752.0
3,Haikyuu!! Karasuno High School vs Shiratorizaw...,TV,10.0,NaN,False,2016.0,2016.0,Fall,"Picking up where the second season ended, the ...",Production I.G,"[Shounen, Sports, Animeism, School Club, S...",[],25134.0,2183,8082,167,4.660,17422.0
4,Attack on Titan 3rd Season: Part II,TV,10.0,NaN,False,2019.0,2019.0,Spring,The battle to retake Wall Maria begins now! Wi...,Wit Studio,"[Action, Fantasy, Horror, Shounen, Dark Fa...","[Cannibalism, Explicit Violence]",21308.0,3217,7864,174,4.650,15789.0
5,Demon Slayer: Kimetsu no Yaiba,TV,26.0,NaN,False,2019.0,2019.0,Spring,"Bloodthirsty demons lurk in the woods, and nob...",ufotable,"[Action, Adventure, Comedy, Drama, Fantasy...",[Violence],28708.0,10844,13156,600,4.647,24765.0
6,Haikyuu!! Second Season,TV,25.0,NaN,False,2015.0,2016.0,Fall,"After losing to Aoba Johsai at the Inter-High,...",Production I.G,"[Shounen, Sports, School Club, School Life,...",[],31329.0,3861,8664,264,4.639,22819.0
7,Hunter x Hunter (2011),TV,148.0,NaN,False,2011.0,2014.0,Fall,"Drawn to the mystique of the unknown, Hunters ...",MADHOUSE,"[Action, Adventure, Drama, Fantasy, Shoune...",[Violence],50195.0,17730,19219,2132,4.635,46853.0
8,Gintama Kanketsu-hen: Yorozuya yo Eien Nare,Movie,1.0,111.0,False,2013.0,2013.0,NaN,"While watching a movie, Gintoki comes upon a ""...",Sunrise,"[Action, Comedy, Drama, Sci Fi, Shounen, ...",[],8454.0,280,6624,150,4.625,6254.0
9,Gintama (2015),TV,51.0,NaN,False,2015.0,2016.0,Spring,Fourth season of Gintama.,BN Pictures,"[Action, Comedy, Drama, Sci Fi, Shounen, ...",[],7862.0,3154,10565,647,4.622,7654.0


In [45]:
#Change year to int instead of double to remove decimal
anime["startYr"] = anime["startYr"].fillna(0.0).astype(int)
anime["finishYr"] = anime["finishYr"].fillna(0.0).astype(int)
anime.head()

,title,mediaType,eps,duration,ongoing,startYr,finishYr,sznOfRelease,description,studios,tags,contentWarn,watched,watching,wantWatch,dropped,rating,votes
0,Fullmetal Alchemist: Brotherhood,TV,64.0,NaN,False,2009,2010,Spring,The foundation of alchemy is based on the law ...,Bones,"[Action, Adventure, Drama, Fantasy, Myster...","[Animal Abuse, Mature Themes, Violence, Dom...",103707.0,14351,25810,2656,4.702,86547.0
1,your name.,Movie,1.0,107.0,False,2016,2016,NaN,Mitsuha and Taki are two total strangers livin...,CoMix Wave Films,"[Drama, Romance, Body Swapping, Gender Bend...",[],58831.0,1453,21733,124,4.663,43960.0
2,A Silent Voice,Movie,1.0,130.0,False,2016,2016,NaN,"After transferring into a new school, a deaf g...",Kyoto Animation,"[Drama, Shounen, Disability, Melancholy, M...","[Bullying, Mature Themes, Suicide]",45892.0,946,17148,132,4.661,33752.0
3,Haikyuu!! Karasuno High School vs Shiratorizaw...,TV,10.0,NaN,False,2016,2016,Fall,"Picking up where the second season ended, the ...",Production I.G,"[Shounen, Sports, Animeism, School Club, S...",[],25134.0,2183,8082,167,4.660,17422.0
4,Attack on Titan 3rd Season: Part II,TV,10.0,NaN,False,2019,2019,Spring,The battle to retake Wall Maria begins now! Wi...,Wit Studio,"[Action, Fantasy, Horror, Shounen, Dark Fa...","[Cannibalism, Explicit Violence]",21308.0,3217,7864,174,4.650,15789.0


In [49]:
anime.sort_values(['rating'], ascending=False)

,title,mediaType,eps,duration,ongoing,startYr,finishYr,sznOfRelease,description,studios,tags,contentWarn,watched,watching,wantWatch,dropped,rating,votes
0,Fullmetal Alchemist: Brotherhood,TV,64.0,NaN,False,2009,2010,Spring,The foundation of alchemy is based on the law ...,Bones,"[Action, Adventure, Drama, Fantasy, Myster...","[Animal Abuse, Mature Themes, Violence, Dom...",103707.0,14351,25810,2656,4.702,86547.0
1,your name.,Movie,1.0,107.0,False,2016,2016,NaN,Mitsuha and Taki are two total strangers livin...,CoMix Wave Films,"[Drama, Romance, Body Swapping, Gender Bend...",[],58831.0,1453,21733,124,4.663,43960.0
2,A Silent Voice,Movie,1.0,130.0,False,2016,2016,NaN,"After transferring into a new school, a deaf g...",Kyoto Animation,"[Drama, Shounen, Disability, Melancholy, M...","[Bullying, Mature Themes, Suicide]",45892.0,946,17148,132,4.661,33752.0
3,Haikyuu!! Karasuno High School vs Shiratorizaw...,TV,10.0,NaN,False,2016,2016,Fall,"Picking up where the second season ended, the ...",Production I.G,"[Shounen, Sports, Animeism, School Club, S...",[],25134.0,2183,8082,167,4.660,17422.0
4,Attack on Titan 3rd Season: Part II,TV,10.0,NaN,False,2019,2019,Spring,The battle to retake Wall Maria begins now! Wi...,Wit Studio,"[Action, Fantasy, Horror, Shounen, Dark Fa...","[Cannibalism, Explicit Violence]",21308.0,3217,7864,174,4.650,15789.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14573,"Welcome to Demon School, Iruma-kun 2",TV,NaN,NaN,False,2021,2021,NaN,"Second Season of Welcome to Demon School, Irum...",,"[Comedy, Shounen, Demons, Monster School, ...",[],0.0,0,1106,0,NaN,NaN
14574,Pinocchio-P: Sekai wa Mada Hajimatte Sura Inai,Music Video,1.0,4.0,False,2020,2020,NaN,NaN,,"[Chibi, Vocaloid]",[],9.0,1,7,1,NaN,NaN
14575,Minagoroshi,OVA,1.0,1.0,False,2005,2005,NaN,NaN,,"[Comedy, Ecchi, No Dialogue, Shorts]",[],3.0,0,7,0,NaN,NaN
14576,Kurayukaba,Movie,NaN,NaN,False,0,0,NaN,NaN,Makaria,[],[],0.0,0,62,0,NaN,NaN


In [52]:
anime['mediaType'].value_counts()

TV             4510
Movie          2512
OVA            2097
Music Video    1596
Web            1555
DVD Special     849
Other           782
TV Special      609
Name: mediaType, dtype: int64

In [62]:
tv_anime = anime.loc[anime['mediaType'] == 'TV'].sort_values('rating', ascending=False)
movie_anime = anime.loc[anime['mediaType'] == 'Movie'].sort_values('rating', ascending=False)
ova_anime = anime.loc[anime['mediaType'] == 'OVA'].sort_values('rating', ascending=False)

In [74]:
tv_anime['sznOfRelease'].value_counts()

Spring    1099
Fall       957
Winter     665
Summer     630
Name: sznOfRelease, dtype: int64